In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = "base_final_emisiones_and_deciles_chile.xlsx" # Nombre del archivo
data = pd.read_excel(path)

In [3]:
def filtrar_df(data, condicion):
    """
    Filtra un DataFrame según una condición dada y realiza cálculos adicionales.
    
    Args:
        data (pd.DataFrame): El DataFrame a procesar.
        condicion (str): La condición a aplicar como cadena. Por ejemplo, "merged_df['dummy_issue'] == 0".
    
    Returns:
        pd.DataFrame: El DataFrame con el resultado de los cálculos.
    """
    # Crear columna 'dummy_issue' basada en la condición de 'issue_count'
    data['dummy_issue'] = np.where(data['issue_count'] > 0, 1, 0)
    
    # Extraer el año de 'Period_End_Date'
    data['Año'] = pd.to_datetime(data['Period_End_Date']).dt.year
    # Función para categorizar el periodo basado en el año
    def categorize_period(year):
        if 2013 <= year <= 2023:
            return '2013-2023'
        else:
            return 'Fuera de rango'
    # Crear la columna 'Period' categorizada
    data['Period'] = data['Año'].apply(categorize_period)
# Filtrar los sector 'Educational Services'
    data = data[data['NAICS_Sector_Name'] != 'Educational Services']
    data = data[data['NAICS_Sector_Name'] != 'Finance and Insurance']
    data = data[data['NAICS_Sector_Name'] != 'Transportation and Warehousing']
    data = data[data['NAICS_Sector_Name'] != 'Administrative and Support and Waste Management and Remediation Services']
    
    
    # Agrupar por Sector y Año para calcular la mediana de ROA
    grouped = data.groupby(['NAICS_Sector_Name', 'Año']).agg(
        median_roa=('ROA', 'median')
    ).reset_index()
    
    # Agregar el periodo al DataFrame agrupado
    grouped['Period'] = grouped['Año'].apply(categorize_period)
    
    # Agrupar por Sector y Periodo para calcular el promedio de las medianas
    benchmark = grouped.groupby(['NAICS_Sector_Name', 'Period']).agg(
        avg_median_roa=('median_roa', 'mean')
    ).reset_index()
    
    # Merge del DataFrame original con el benchmark
    merged_df = pd.merge(data, benchmark, on=['NAICS_Sector_Name', 'Period'], how='inner')
    
    # Calcular las utilidades típicas y el excedente de la firma
    merged_df['tipical_profits'] = merged_df['Total_Assets,_Reported'] * merged_df['avg_median_roa']
    
    # Agrupar por Sector y Año para calcular la mediana de ROA
    grouped = merged_df.groupby(['NAICS_Sector_Name', 'Period']).agg(
        prom_tipical=('tipical_profits', 'median')
    ).reset_index()
    
    # Merge del DataFrame original con el benchmark
    merged_df = pd.merge(merged_df , grouped, on=['NAICS_Sector_Name', 'Period'], how='inner')
    
    # Filtrar el DataFrame usando la condición proporcionada
    merged_df = merged_df.loc[eval(condicion)]
    
    merged_df['firm_surplus'] = (merged_df['Net_Income_Before_Taxes'] - merged_df['tipical_profits'])
    
    # Agrupar por Periodo para calcular el numerador y denominador del excedente
    surplus = merged_df.groupby(['NAICS_Sector_Name','Period']).agg(
        numerador=('firm_surplus', 'sum'),
        denominador=('Net_Income_Before_Taxes', 'sum')
    ).reset_index()
    
    # Calcular el resultado como porcentaje
    surplus['result'] = (surplus['numerador'] / surplus['denominador']) * 100
    
    calc = merged_df.groupby(['NAICS_Sector_Name', 'Period']).agg(
        prom_tipical=('firm_surplus', 'mean'),
        net_inc=('Net_Income_Before_Taxes', 'mean'),
    ).reset_index()

    
    calc['difference'] = (calc['net_inc'] - calc['prom_tipical']) / (calc['net_inc'])
    
    return calc
    

In [25]:
surplus1 =filtrar_df(data, "merged_df['dummy_issue'] ==0")
surplus2 =filtrar_df(data, "merged_df['dummy_issue'] ==1")
surplus2 = surplus2.rename(columns={'difference': 'difference2'})
surplus1 = surplus1.drop(columns=['Period', 'prom_tipical', 'net_inc'])
surplus2 = surplus2.drop(columns=['Period', 'prom_tipical', 'net_inc'])

# Merge del DataFrame original con el benchmark
merged = pd.merge( surplus1, surplus2, on=['NAICS_Sector_Name'], how='outer')

merged['diff'] = (merged['difference'] - merged['difference2']) 

latex_code = merged.to_latex(index=False, escape=False,float_format="%.4f")
print(latex_code)

\begin{tabular}{lrrr}
\toprule
NAICS_Sector_Name & difference & difference2 & diff \\
\midrule
Accommodation and Food Services & -0.1163 & 0.2280 & -0.3443 \\
Agriculture, Forestry, Fishing and Hunting & 0.7633 & NaN & NaN \\
Arts, Entertainment, and Recreation & -0.3615 & NaN & NaN \\
Construction & 0.8298 & NaN & NaN \\
Health Care and Social Assistance & 1.4375 & NaN & NaN \\
Information & 0.9469 & 0.9851 & -0.0382 \\
Manufacturing & 0.8047 & 0.3433 & 0.4614 \\
Mining, Quarrying, and Oil and Gas Extraction & -0.0447 & -0.0610 & 0.0163 \\
Other Services (except Public Administration) & 0.4254 & NaN & NaN \\
Professional, Scientific, and Technical Services & 0.6856 & NaN & NaN \\
Real Estate and Rental and Leasing & -0.1637 & NaN & NaN \\
Retail Trade & 0.9357 & 0.9432 & -0.0075 \\
Utilities & 0.8492 & 0.6977 & 0.1516 \\
Wholesale Trade & 0.7756 & NaN & NaN \\
\bottomrule
\end{tabular}



C:\Users\rober\AppData\Local\Temp\ipykernel_8972\1750495961.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['firm_surplus'] = (merged_df['Net_Income_Before_Taxes'] - merged_df['tipical_profits'])
C:\Users\rober\AppData\Local\Temp\ipykernel_8972\1750495961.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['firm_surplus'] = (merged_df['Net_Income_Before_Taxes'] - merged_df['tipical_profits'])


In [ ]:
def filtrar_df2(data, condicion):
    """
    Filtra un DataFrame según una condición dada y realiza cálculos adicionales.
    
    Args:
        data (pd.DataFrame): El DataFrame a procesar.
        condicion (str): La condición a aplicar como cadena. Por ejemplo, "merged_df['dummy_issue'] == 0".
    
    Returns:
        pd.DataFrame: El DataFrame con el resultado de los cálculos.
    """
    # Crear columna 'dummy_issue' basada en la condición de 'issue_count'
    data['dummy_issue'] = np.where(data['issue_count'] > 0, 1, 0)
    
    # Extraer el año de 'Period_End_Date'
    data['Año'] = pd.to_datetime(data['Period_End_Date']).dt.year
    # Función para categorizar el periodo basado en el año
    def categorize_period(year):
        if 2013 <= year <= 2023:
            return '2013-2023'
        else:
            return 'Fuera de rango'
    # Crear la columna 'Period' categorizada
    data['Period'] = data['Año'].apply(categorize_period)
# Filtrar los sector 'Educational Services'
    data = data[data['NAICS_Sector_Name'] != 'Educational Services']
    data = data[data['NAICS_Sector_Name'] != 'Finance and Insurance']
    data = data[data['NAICS_Sector_Name'] != 'Transportation and Warehousing']
    data = data[data['NAICS_Sector_Name'] != 'Administrative and Support and Waste Management and Remediation Services']
    
    
    # Agrupar por Sector y Año para calcular la mediana de ROA
    grouped = data.groupby(['NAICS_Sector_Name', 'Año']).agg(
        median_roa=('ROA', 'median')
    ).reset_index()
    
    # Agregar el periodo al DataFrame agrupado
    grouped['Period'] = grouped['Año'].apply(categorize_period)
    
    # Agrupar por Sector y Periodo para calcular el promedio de las medianas
    benchmark = grouped.groupby(['NAICS_Sector_Name', 'Period']).agg(
        avg_median_roa=('median_roa', 'mean')
    ).reset_index()
    
    # Merge del DataFrame original con el benchmark
    merged_df = pd.merge(data, benchmark, on=['NAICS_Sector_Name', 'Period'], how='inner')
    
    # Calcular las utilidades típicas y el excedente de la firma
    merged_df['tipical_profits'] = merged_df['Total_Assets,_Reported'] * merged_df['avg_median_roa']
    
    # Agrupar por Sector y Año para calcular la mediana de ROA
    grouped = merged_df.groupby(['NAICS_Sector_Name', 'Period']).agg(
        prom_tipical=('tipical_profits', 'median')
    ).reset_index()
    
    # Merge del DataFrame original con el benchmark
    merged_df = pd.merge(merged_df , grouped, on=['NAICS_Sector_Name', 'Period'], how='inner')
    
    # Filtrar el DataFrame usando la condición proporcionada
    merged_df = merged_df.loc[eval(condicion)]
    
    merged_df['firm_surplus'] = (merged_df['Net_Income_Before_Taxes'] - merged_df['tipical_profits'])
    
    # Agrupar por Periodo para calcular el numerador y denominador del excedente
    surplus = merged_df.groupby(['NAICS_Sector_Name','Period']).agg(
        tipical_mean=('tipical_profits', 'mean'),
        net_mean=('Net_Income_Before_Taxes', 'mean')
    ).reset_index()
    
    # Calcular el resultado como porcentaje
    surplus['Difference'] = ((surplus['tipical_mean'] - surplus['tipical_mean'])/ surplus['net_mean']) * 100
    
    return surplus
    

In [10]:
surplus1 =filtrar_df2(data, "merged_df['dummy_issue'] ==0")
surplus2 =filtrar_df2(data, "merged_df['dummy_issue'] ==1")
surplus2 = surplus2.rename(columns={'result': 'result2'})
surplus1 = surplus1.drop(columns=['Period', 'tipical_mean', 'net_mean'])
surplus2 = surplus2.drop(columns=['Period', 'tipical_mean', 'net_mean'])

# Merge del DataFrame original con el benchmark
merged = pd.merge( surplus1, surplus2, on=['NAICS_Sector_Name'], how='outer')

merged['difference'] = (merged['result'] - merged['result2']) 

latex_code = merged.to_latex(index=False, escape=False,float_format="%.4f")
print(latex_code)

\begin{tabular}{lrrr}
\toprule
NAICS_Sector_Name & result & result2 & difference \\
\midrule
Accommodation and Food Services & 111.6295 & 77.1957 & 34.4338 \\
Agriculture, Forestry, Fishing and Hunting & 23.6663 & NaN & NaN \\
Arts, Entertainment, and Recreation & 136.1549 & NaN & NaN \\
Construction & 17.0181 & NaN & NaN \\
Health Care and Social Assistance & -43.7543 & NaN & NaN \\
Information & 5.3132 & 1.4911 & 3.8221 \\
Manufacturing & 19.5301 & 65.6745 & -46.1444 \\
Mining, Quarrying, and Oil and Gas Extraction & 104.4669 & 106.1010 & -1.6340 \\
Other Services (except Public Administration) & 57.4582 & NaN & NaN \\
Professional, Scientific, and Technical Services & 31.4430 & NaN & NaN \\
Real Estate and Rental and Leasing & 116.3699 & NaN & NaN \\
Retail Trade & 6.4269 & 5.6771 & 0.7498 \\
Utilities & 15.0754 & 30.2320 & -15.1566 \\
Wholesale Trade & 22.4398 & NaN & NaN \\
\bottomrule
\end{tabular}



C:\Users\rober\AppData\Local\Temp\ipykernel_12024\2509578770.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['firm_surplus'] = (merged_df['Net_Income_Before_Taxes'] - merged_df['tipical_profits'])
C:\Users\rober\AppData\Local\Temp\ipykernel_12024\2509578770.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['firm_surplus'] = (merged_df['Net_Income_Before_Taxes'] - merged_df['tipical_profits'])


In [20]:
def filtrar_df3(data, condicion):
    """
    Filtra un DataFrame según una condición dada y realiza cálculos adicionales.
    
    Args:
        data (pd.DataFrame): El DataFrame a procesar.
        condicion (str): La condición a aplicar como cadena. Por ejemplo, "merged_df['dummy_issue'] == 0".
    
    Returns:
        pd.DataFrame: El DataFrame con el resultado de los cálculos.
    """
    # Crear columna 'dummy_issue' basada en la condición de 'issue_count'
    data['dummy_issue'] = np.where(data['issue_count'] > 0, 1, 0)
    
    # Extraer el año de 'Period_End_Date'
    data['Año'] = pd.to_datetime(data['Period_End_Date']).dt.year
    # Función para categorizar el periodo basado en el año
    def categorize_period(year):
        if 2013 <= year <= 2023:
            return '2013-2023'
        else:
            return 'Fuera de rango'
    # Crear la columna 'Period' categorizada
    data['Period'] = data['Año'].apply(categorize_period)
# Filtrar los sector 'Educational Services'
    data = data[data['NAICS_Sector_Name'] != 'Educational Services']
    data = data[data['NAICS_Sector_Name'] != 'Finance and Insurance']
    data = data[data['NAICS_Sector_Name'] != 'Transportation and Warehousing']
    data = data[data['NAICS_Sector_Name'] != 'Administrative and Support and Waste Management and Remediation Services']
    
    
    # Agrupar por Sector y Año para calcular la mediana de ROA
    grouped = data.groupby(['NAICS_Sector_Name', 'Año']).agg(
        median_roa=('ROA', 'median')
    ).reset_index()
    
    # Agregar el periodo al DataFrame agrupado
    grouped['Period'] = grouped['Año'].apply(categorize_period)
    
    # Agrupar por Sector y Periodo para calcular el promedio de las medianas
    benchmark = grouped.groupby(['NAICS_Sector_Name', 'Period']).agg(
        avg_median_roa=('median_roa', 'mean')
    ).reset_index()
    
    # Merge del DataFrame original con el benchmark
    merged_df = pd.merge(data, benchmark, on=['NAICS_Sector_Name', 'Period'], how='inner')
    
    # Calcular las utilidades típicas y el excedente de la firma
    merged_df['tipical_profits'] = merged_df['Total_Assets,_Reported'] * merged_df['avg_median_roa']
    
    # Agrupar por Sector y Año para calcular la mediana de ROA
    grouped = merged_df.groupby(['NAICS_Sector_Name', 'Period']).agg(
        prom_tipical=('tipical_profits', 'mean')
    ).reset_index()
    
    # Merge del DataFrame original con el benchmark
    merged_df = pd.merge(merged_df , grouped, on=['NAICS_Sector_Name', 'Period'], how='inner')
    
    # Filtrar el DataFrame usando la condición proporcionada
    merged_df = merged_df.loc[eval(condicion)]
    
    merged_df['firm_surplus'] = (merged_df['Net_Income_Before_Taxes'] - merged_df['prom_tipical'])
    
    # Agrupar por Periodo para calcular el numerador y denominador del excedente
    surplus = merged_df.groupby(['NAICS_Sector_Name','Period']).agg(
        tipical_mean=('prom_tipical', 'mean'),
        net_mean=('Net_Income_Before_Taxes', 'mean')
    ).reset_index()
    
    # Calcular el resultado como porcentaje
    surplus['result'] = ((surplus['net_mean'] - surplus['tipical_mean'])/ surplus['net_mean']) * 100
    
    return surplus
    

In [21]:
surplus1 =filtrar_df3(data, "merged_df['dummy_issue'] ==0")
surplus2 =filtrar_df3(data, "merged_df['dummy_issue'] ==1")
surplus2 = surplus2.rename(columns={'result': 'result2'})
surplus1 = surplus1.drop(columns=['Period', 'tipical_mean', 'net_mean'])
surplus2 = surplus2.drop(columns=['Period', 'tipical_mean', 'net_mean'])
# Merge del DataFrame original con el benchmark
merged = pd.merge( surplus1, surplus2, on=['NAICS_Sector_Name'], how='outer')

merged['difference'] = (merged['result'] - merged['result2']) 

latex_code = merged.to_latex(index=False, escape=False,float_format="%.4f")
print(latex_code)


\begin{tabular}{lrrr}
\toprule
NAICS_Sector_Name & result & result2 & difference \\
\midrule
Accommodation and Food Services & 116065711.9415 & 92.3986 & 116065619.5429 \\
Agriculture, Forestry, Fishing and Hunting & 23.6663 & NaN & NaN \\
Arts, Entertainment, and Recreation & 136.1549 & NaN & NaN \\
Construction & 17.0181 & NaN & NaN \\
Health Care and Social Assistance & -43.7543 & NaN & NaN \\
Information & -41.4602 & 72.9756 & -114.4358 \\
Manufacturing & -2.7595 & 85.7631 & -88.5225 \\
Mining, Quarrying, and Oil and Gas Extraction & 133.4287 & 101.5587 & 31.8700 \\
Other Services (except Public Administration) & 57.4582 & NaN & NaN \\
Professional, Scientific, and Technical Services & 31.4430 & NaN & NaN \\
Real Estate and Rental and Leasing & 116.3699 & NaN & NaN \\
Retail Trade & -222.8756 & 71.3476 & -294.2232 \\
Utilities & -108.9008 & 79.0790 & -187.9797 \\
Wholesale Trade & 22.4398 & NaN & NaN \\
\bottomrule
\end{tabular}



C:\Users\rober\AppData\Local\Temp\ipykernel_12024\1132308514.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['firm_surplus'] = (merged_df['Net_Income_Before_Taxes'] - merged_df['prom_tipical'])
C:\Users\rober\AppData\Local\Temp\ipykernel_12024\1132308514.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['firm_surplus'] = (merged_df['Net_Income_Before_Taxes'] - merged_df['prom_tipical'])
